In [ ]:
import csv 
import json
 
import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config


all_segments = {}
for config in mesh_config.mesh_configs:
    mesh = wavefront.load_obj(config.mesh)
    with open(config.output_csv, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        segments=[]

        for row in reader:
            uid = int(row[0])
            name = row[1]
            num_leds = int(row[2])
            length = float(row[3])
            # reversed = row[4]
            
            for group in mesh.groups:
                if group.name == name:
                    break

            group = next((g for g in mesh.groups if g.name == name), None)
            if not group:
                print("Couldn't find mesh group for %s" % name)
                continue

            data = led_config_utils.prepare_data(group)
            data_2d = led_config_utils.project_2d(data)
            clusters, labels = led_config_utils.cluster(
                data_2d, eps=config.cluster_eps, min_samples=3)

            nodes = led_config_utils.create_line_segments(data, clusters, labels)
            if nodes == None:
                continue
            leds_distance = length / num_leds
            points = led_config_utils.trace_line_segments(nodes, num_leds, config.start_offset, leds_distance)
            print('Segment %s: length=%.1fm, num_clusters=%s, num_leds=%s' %
                (group.name, length, len(clusters), len(points)))
            if len(points) == 0:
                continue
            segment = led_config.Segment(
                uid=uid, name=group.name, points=points, num_leds=points.shape[0], length=length)
            segments.append(segment)
            all_segments[uid] = segment
            # led_config_utils.plot_segment(data, clusters, labels, points)

# Collapse some segments
UIDS_TRUNK = [101,102,103,104,105,106,107,108,109,110,111,112,113,114,115]
UIDS_TUSKS_RIGHT = [116,117,118,119,120,121,122,123,124,125,126]
UIDS_TUSKS_LEFT = [127,128,129,130,131,132,133,134,135,136,137]
for merge_list in [UIDS_TRUNK, UIDS_TUSKS_RIGHT, UIDS_TUSKS_LEFT]:
    merged_segment = all_segments[merge_list[0]]
    for uid in merge_list[1:]:
        segment = all_segments.pop(uid)
        merged_segment.merge(segment)

# Create LED config
config = led_config.LedConfig()
for segment in all_segments.values():
    config.led_segments.append(segment)
    config.total_num_segments += 1
    config.total_length += segment.length
    config.total_num_leds += segment.num_leds

with open('../config/led_config.json', 'w', encoding='utf-8') as f:
    json.dump(config.to_dict(), f, ensure_ascii=False, indent=4)

